
# DL-Fall-25 — Llama3-8B SFT (QLoRA) — End-to-End Notebook (v3, mac-or-kaggle)

**Why this version?** Your last run was on **macOS/Apple Silicon (no CUDA)**, so `bitsandbytes` couldn't install.  
This notebook **auto-detects** the environment:

- **Linux + NVIDIA GPU** → uses **bitsandbytes QLoRA** (recommended for Llama-3-8B).  
- **No CUDA (macOS/CPU)** → **falls back** to a tiny open model so you can **smoke-test the pipeline**.  
  For leaderboard runs, move to **Kaggle/Colab/AWS (Linux GPU)**.


## 0) Quick environment check

In [6]:
import platform, torch
print("Python :", platform.python_version())
print("OS     :", platform.platform())
print("CUDA   :", torch.version.cuda)
print("GPU    :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "NO CUDA GPU")


Python : 3.13.3
OS     : macOS-15.6.1-arm64-arm-64bit-Mach-O
CUDA   : None
GPU    : NO CUDA GPU


In [7]:
%pip -q install -U "accelerate>=0.33.0" "datasets>=2.20.0" "evaluate>=0.4.2" \
"numpy>=1.26.0" "pandas>=2.1.0" "peft>=0.12.0" "scikit-learn>=1.3.0" \
"transformers>=4.44.0" "trl>=0.10.1"



[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1) Config (with mac-safe fallback)

In [8]:
from dataclasses import dataclass

@dataclass
class Config:
    # Small, fully open model so macOS can run end-to-end without tokens/GPU.
    # (Later on Kaggle GPU, switch to Llama-3-8B — see Kaggle cells at bottom.)
    model_id: str = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

    out_dir: str = "outputs"
    use_solution: int = 1           # 1 = include 'solution' text in the prompt
    epochs: int = 1
    lr: float = 2e-4
    per_device_bs: int = 1          # safe defaults
    grad_accum: int = 8
    max_len: int = 1024             # smaller for mac speed
    seed: int = 42
    report_to: str = "none"

cfg = Config()
cfg


Config(model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0', out_dir='outputs', use_solution=1, epochs=1, lr=0.0002, per_device_bs=1, grad_accum=8, max_len=1024, seed=42, report_to='none')

## 2) Install deps

In [3]:

# Use %pip (IPython magic) to avoid zsh parsing issues.
if HAS_CUDA:
    %pip -q install -U "accelerate>=0.33.0" "bitsandbytes>=0.43.0"     "datasets>=2.20.0" "evaluate>=0.4.2" "numpy>=1.26.0" "pandas>=2.1.0"     "peft>=0.12.0" "scikit-learn>=1.3.0" "transformers>=4.44.0" "trl>=0.10.1"
else:
    print("No CUDA detected — installing CPU/MPS-safe deps (skipping bitsandbytes).")
    %pip -q install -U "accelerate>=0.33.0"     "datasets>=2.20.0" "evaluate>=0.4.2" "numpy>=1.26.0" "pandas>=2.1.0"     "peft>=0.12.0" "scikit-learn>=1.3.0" "transformers>=4.44.0" "trl>=0.10.1"


No CUDA detected — installing CPU/MPS-safe deps (skipping bitsandbytes).
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.2 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip

SyntaxError: invalid syntax (3767263527.py, line 1)


> **Note**  
> - `bitsandbytes` requires **Linux + NVIDIA GPU**.  
> - On macOS (M1/M2/M3), use **Kaggle/Colab/AWS** for the real Llama-3-8B run.  
> - This notebook will **auto-switch** to a small model locally so you can test the pipeline.


## 3) Hugging Face login (optional if using public fallback model)

In [4]:

from huggingface_hub import login
import os

def optional_hf_login():
    tok = os.getenv("HF_TOKEN")
    if tok:
        try:
            login(tok); print("HF login OK."); return True
        except Exception as e:
            print("HF login failed:", e)
    else:
        print("No HF_TOKEN environment variable found. If your model is gated (e.g., Llama-3), set HF_TOKEN.")
    return False

LOGGED_IN = optional_hf_login()


No HF_TOKEN environment variable found. If your model is gated (e.g., Llama-3), set HF_TOKEN.


/opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 4) Dataset

In [10]:
from datasets import load_dataset
ds = load_dataset("ad6398/nyu-dl-teach-maths-comp")
train_ds, test_ds = ds["train"], ds["test"]


ImportError: cannot import name '_slice' from 'numpy._core.umath' (/opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/lib/python3.13/site-packages/numpy/_core/umath.py)

In [11]:
import sys, pkgutil, numpy
print("Python:", sys.version)
print("NumPy :", numpy.__version__)
print("Has SciPy installed? ", any(m.name == "scipy" for m in pkgutil.iter_modules()))


Python: 3.13.3 (main, Apr  8 2025, 13:54:08) [Clang 16.0.0 (clang-1600.0.26.6)]
NumPy : 2.2.2
Has SciPy installed?  True


## 5) Utilities (prompting + constrained decoding)

In [12]:
# Remove SciPy so `datasets` doesn't try to patch it (the patch fails on Py3.13 macOS)
%pip -q uninstall -y scipy
import sys
# Clean any partially-imported modules from sys.modules
for k in list(sys.modules.keys()):
    if k.startswith("scipy"):
        sys.modules.pop(k, None)
print("✅ SciPy uninstalled/cleared for this kernel.")


Note: you may need to restart the kernel to use updated packages.
✅ SciPy uninstalled/cleared for this kernel.


In [13]:
from datasets import load_dataset
ds = load_dataset("ad6398/nyu-dl-teach-maths-comp")  # public; no token needed
train_ds, test_ds = ds["train"], ds["test"]
print(train_ds)
print(test_ds)


Generating train split: 100%|█| 1000000/1000000 [00:00<00:00, 1860932.65 example
Generating test split: 100%|██| 10000/10000 [00:00<00:00, 1360726.71 examples/s]


Dataset({
    features: ['question', 'is_correct', 'answer', 'solution'],
    num_rows: 1000000
})
Dataset({
    features: ['question', 'is_correct', 'answer', 'solution'],
    num_rows: 10000
})


In [14]:
import random, numpy as np
from transformers import LogitsProcessor

def seed_everything(seed: int = 42):
    random.seed(seed); np.random.seed(seed)

SYS_PROMPT = (
    "You are a precise math answer verifier. "
    "Given a problem, a student's answer, and an optional worked solution, "
    "respond with a single digit label without explanation:\n"
    "- 1 if the student's answer is correct\n"
    "- 0 if the student's answer is incorrect"
)

def build_user_block(example, use_solution: bool = True) -> str:
    q = (example.get("question") or "").strip()
    a = (example.get("answer") or "").strip()
    sol = example.get("solution") or ""
    parts = [f"Problem:\n{q}\n\nStudent Answer:\n{a}"]
    if use_solution and isinstance(sol, str) and sol.strip():
        parts.append(f"\nSolution (reference reasoning):\n{sol.strip()}")
    parts.append("\nReturn only the digit (1 or 0).")
    return "\n".join(parts)

def make_text(example, use_solution: bool, label=None):
    # Simple, model-agnostic format (avoids chat-template differences)
    hdr = f"<<SYS>>\n{SYS_PROMPT}\n<</SYS>>\n\n"
    usr = f"<<USR>>\n{build_user_block(example, use_solution)}\n<</USR>>\n\n"
    asst = "<<ASSISTANT>>\n" + (str(int(label)) if label is not None else "")
    return hdr + usr + asst

class AllowedTokensLogitsProcessor(LogitsProcessor):
    def __init__(self, allowed_token_ids): self.allowed = set(allowed_token_ids)
    def __call__(self, input_ids, scores):
        import torch
        mask = torch.full_like(scores, float("-inf"))
        for tid in self.allowed:
            if tid < scores.shape[-1]:
                mask[..., tid] = 0.0
        return scores + mask

def get_allowed_token_ids(tokenizer):
    return [tokenizer.convert_tokens_to_ids(t) for t in ["0", "1"]]


In [20]:
from datasets import DatasetDict

seed_everything(cfg.seed)
os.makedirs(cfg.out_dir, exist_ok=True)

# Make is_correct a ClassLabel (overwrites train_ds)
train_ds = train_ds.class_encode_column("is_correct")

dd = train_ds.train_test_split(
    test_size=0.05, 
    seed=cfg.seed, 
    stratify_by_column="is_correct"
)

# Optional subsample for speed
if cfg.TRAIN_LIMIT:
    dd["train"] = dd["train"].shuffle(seed=cfg.seed).select(range(cfg.TRAIN_LIMIT))
if cfg.VAL_LIMIT:
    dd["test"]  = dd["test"].shuffle(seed=cfg.seed).select(range(cfg.VAL_LIMIT))

dataset = DatasetDict({"train": dd["train"], "validation": dd["test"]})
dataset


DatasetDict({
    train: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 500
    })
})

In [21]:
import os, torch
seed_everything(cfg.seed)
os.makedirs(cfg.out_dir, exist_ok=True)

dd = train_ds.train_test_split(test_size=0.05, seed=cfg.seed, stratify_by_column="is_correct")
if cfg.TRAIN_LIMIT:
    dd["train"] = dd["train"].shuffle(seed=cfg.seed).select(range(cfg.TRAIN_LIMIT))
if cfg.VAL_LIMIT:
    dd["test"]  = dd["test"].shuffle(seed=cfg.seed).select(range(cfg.VAL_LIMIT))
dataset = DatasetDict({"train": dd["train"], "validation": dd["test"]})
dataset


DatasetDict({
    train: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 500
    })
})

In [23]:
# 1) Remove scikit-learn so Transformers won't try to import it (which needs scipy)
%pip -q uninstall -y scikit-learn

# 2) Clear any already-imported sklearn modules from the running process
import sys
for k in list(sys.modules.keys()):
    if k.startswith("sklearn"):
        sys.modules.pop(k, None)

print("✅ scikit-learn removed from this kernel.")


Note: you may need to restart the kernel to use updated packages.
✅ scikit-learn removed from this kernel.


In [25]:
# Fallback: pin versions that avoid the sklearn path during import
%pip -q install -U "transformers==4.40.2" "trl==0.9.6" "peft>=0.12.0"
# then import as usual
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch
print("✅ transformers pinned + imported.")


  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [99 lines of output]
      Running `maturin pep517 build-wheel -i /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python --compatibility off`
      Python reports SOABI: cpython-313-darwin
      Computed rustc target triple: aarch64-apple-darwin
      Installation directory: /Users/yuktakulkarni/Library/Caches/puccinialin
      Rustup already downloaded
      Installing rust to /Users/yuktakulkarni/Library/Caches/puccinialin/rustup
      warn: It looks like you have an existing rustup settings file at:
      warn: /Users/yuktakulkarni/.rustup/settings.toml
      warn: Rustup will install the default toolchain as specified in the settings file,
      warn: instead of the one inferred from the default host triple.
      info: profile set to 'minimal'
      info: default host triple is aarch64-apple-darwin
      warn: Updating existing toolchain,

ModuleNotFoundError: Could not import module 'AutoTokenizer'. Are this object's requirements defined correctly?

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch

# TinyLlama (from your cfg earlier)
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
tokenizer.padding_side = "right"
tokenizer.truncation_side = "left"

device_map = "mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available() else "auto"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map=device_map)

peft_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05, bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)
model = get_peft_model(model, peft_cfg)
model.print_trainable_parameters()

def preprocess(example, use_solution: bool = True):
    label = 1 if example["is_correct"] else 0
    # same make_text you already defined earlier
    text = make_text(example, use_solution=bool(use_solution), label=label)
    return {"text": text}

proc_train = dataset["train"].map(lambda x: preprocess(x, True))
proc_val   = dataset["validation"].map(lambda x: preprocess(x, True))
len(proc_train), len(proc_val)


## 7) Inference → `submission.csv`

In [ ]:

import os, torch, pandas as pd
from transformers import AutoModelForCausalLM, GenerationConfig
from peft import PeftModel

model_id_infer = model_id  # from training cell above

if use_bnb:
    from transformers import BitsAndBytesConfig
    quant_cfg2 = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=dtype if dtype is not None else torch.float16,
    )
    base = AutoModelForCausalLM.from_pretrained(
        model_id_infer, quantization_config=quant_cfg2, device_map="auto", torch_dtype=dtype if dtype is not None else torch.float16
    )
else:
    device_map = "mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available() else "auto"
    base = AutoModelForCausalLM.from_pretrained(model_id_infer, device_map=device_map)

model_inf = PeftModel.from_pretrained(base, os.path.join(cfg.out_dir, "adapter"))
model_inf.eval()

allowed_ids = get_allowed_token_ids(tokenizer)
logits_proc = AllowedTokensLogitsProcessor(allowed_ids)

gen_cfg = GenerationConfig(
    max_new_tokens=1, do_sample=False, temperature=0.0, top_p=1.0,
    eos_token_id=[tokenizer.eos_token_id], pad_token_id=tokenizer.eos_token_id,
)

preds = []
for i, ex in enumerate(test_ds):
    msgs = to_chat_messages(ex, use_solution=bool(cfg.use_solution), label=None)
    prompt = tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=cfg.max_len).to(model_inf.device)
    with torch.no_grad():
        out = model_inf.generate(**inputs, generation_config=gen_cfg, logits_processor=[logits_proc])
    new_tokens = out[0, inputs["input_ids"].shape[1]:]
    text = tokenizer.decode(new_tokens, skip_special_tokens=True)
    ch = (text.strip()[:1] if text.strip() else "0")
    is_correct = True if ch == "1" else False
    preds.append({"ID": i, "is_correct": is_correct})
    if (i+1) % 500 == 0:
        print(f"Predicted {i+1} / {len(test_ds)}")

import pandas as pd
sub = pd.DataFrame(preds, columns=["ID", "is_correct"])
os.makedirs(cfg.out_dir, exist_ok=True)
sub_path = os.path.join(cfg.out_dir, "submission.csv")
sub.to_csv(sub_path, index=False)
print(f"Saved submission to: {sub_path}")

if os.environ.get("KAGGLE_URL_BASE"):
    sub_kaggle = "/kaggle/working/submission.csv"
    sub.to_csv(sub_kaggle, index=False)
    print(f"Also wrote {sub_kaggle}")


## 8) Validate format

In [ ]:

import pandas as pd, os
sub_path = os.path.join(cfg.out_dir, "submission.csv")
df = pd.read_csv(sub_path)
display(df.head())
print("Rows:", len(df), " Unique IDs:", df["ID"].nunique())
print("Value counts:", df["is_correct"].value_counts(dropna=False))
assert set(df["is_correct"].unique()).issubset({True, False})
assert df["ID"].nunique() == len(df)
print("✅ Format checks passed.")



### Running on Kaggle (recommended for Llama-3-8B)
1. Create a **Kaggle Notebook** → **Settings** → Turn **Internet: On**, **GPU: T4**.  
2. Add a **Secret** named `HF_TOKEN` with your Hugging Face token.  
3. Upload this notebook and run top-to-bottom.  
4. `submission.csv` will appear under `/kaggle/working/` for direct submission.
